# DEMO - Training and testing HerdNet on nadir aerial images

## Installations

In [1]:
!nvidia-smi

Wed Aug 14 09:53:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     Off | 00000000:22:00.0 Off |                  N/A |
|  0%   26C    P8               7W / 280W |      2MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# # Install the dependencies
# !pip install albumentations==1.0.3
# !pip install fiftyone==0.14.3
# !pip install hydra-core==1.1.0
# !pip install opencv-python==4.5.1.48
# !pip install pandas==1.2.3
# !pip install pillow==8.2.0
# !pip install scikit-image==0.18.1
# !pip install scikit-learn==1.0.2
# !pip install scipy==1.6.2
# !pip install wandb==0.10.33

In [3]:
# Download and install the code
import sys
# 
# !git clone https://github.com/Alexandre-Delplanque/HerdNet
# !cd '/content/HerdNet' && python setup.py install
# 
# sys.path.append('/content/HerdNet')

## Create datasets

In [6]:
# Download some of the data of Delplanque et al. (2021) as an example
# !gdown 1CcTAZZJdwrBfCPJtVH6VBU3luGKIN9st -O ./data/data.zip
# !unzip -oq "../data/data.zip" -d ../

In [2]:
# Set the seed
from animaloc.utils.seed import set_seed

set_seed(9292)

/home/christian/anaconda3/envs/HerdNet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Create validation patches using the patcher tool (for demo)
from animaloc.utils.useful_funcs import mkdir

mkdir('../data/val_patches')
# !python ../tools/patcher.py ../data/val 512 512 0 ../data/val_patches -csv ../data/val.csv -min 0.0 -all False

Exporting patches: 100%|██████████████████████| 111/111 [02:04<00:00,  1.12s/it]


In [8]:
# Training, validation and test datasets
import albumentations as A

from animaloc.datasets import CSVDataset
from animaloc.data.transforms import MultiTransformsWrapper, DownSample, PointsToMask, FIDT

patch_size = 512
num_classes = 7
down_ratio = 2

train_dataset = CSVDataset(
    csv_file = '../data/train_patches.csv',
    root_dir = '../data/train_patches',
    albu_transforms = [
        A.VerticalFlip(p=0.5), 
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.2),
        A.Blur(blur_limit=15, p=0.2),
        A.Normalize(p=1.0)
        ],
    end_transforms = [MultiTransformsWrapper([
        FIDT(num_classes=num_classes, down_ratio=down_ratio),
        PointsToMask(radius=2, num_classes=num_classes, squeeze=True, down_ratio=int(patch_size//16))
        ])]
    )

val_dataset = CSVDataset(
    csv_file = '../data/val_patches/gt.csv',
    root_dir = '../data/val_patches',
    albu_transforms = [A.Normalize(p=1.0)],
    end_transforms = [DownSample(down_ratio=down_ratio, anno_type='point')]
    )

test_dataset = CSVDataset(
    csv_file = '../data/test.csv',
    root_dir = '../data/test',
    albu_transforms = [A.Normalize(p=1.0)],
    end_transforms = [DownSample(down_ratio=down_ratio, anno_type='point')]
    )

In [16]:
# Dataloaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset = train_dataset, batch_size = 10, shuffle = True)

val_dataloader = DataLoader(dataset = val_dataset, batch_size = 1, shuffle = False)

test_dataloader = DataLoader(dataset = test_dataset, batch_size = 1, shuffle = False)

## Define HerdNet for training

In [17]:
from animaloc.models import HerdNet
from torch import Tensor
from animaloc.models import LossWrapper
from animaloc.train.losses import FocalLoss
from torch.nn import CrossEntropyLoss

herdnet = HerdNet(num_classes=num_classes, down_ratio=down_ratio).cuda()    

weight = Tensor([0.1, 1.0, 2.0, 1.0, 6.0, 12.0, 1.0]).cuda()

losses = [
    {'loss': FocalLoss(reduction='mean'), 'idx': 0, 'idy': 0, 'lambda': 1.0, 'name': 'focal_loss'},
    {'loss': CrossEntropyLoss(reduction='mean', weight=weight), 'idx': 1, 'idy': 1, 'lambda': 1.0, 'name': 'ce_loss'}
    ]

herdnet = LossWrapper(herdnet, losses=losses)

## Create the Trainer

In [18]:
from torch.optim import Adam
from animaloc.train import Trainer
from animaloc.eval import PointsMetrics, HerdNetStitcher, HerdNetEvaluator
from animaloc.utils.useful_funcs import mkdir

work_dir = '../output'
mkdir(work_dir)

lr = 1e-4
weight_decay = 1e-3
epochs = 100

optimizer = Adam(params=herdnet.parameters(), lr=lr, weight_decay=weight_decay)

metrics = PointsMetrics(radius=20, num_classes=num_classes)

stitcher = HerdNetStitcher(
    model=herdnet, 
    size=(patch_size,patch_size), 
    overlap=160, 
    down_ratio=down_ratio, 
    reduction='mean'
    )

evaluator = HerdNetEvaluator(
    model=herdnet, 
    dataloader=val_dataloader, 
    metrics=metrics, 
    stitcher=stitcher, 
    work_dir=work_dir, 
    header='validation'
    )

trainer = Trainer(
    model=herdnet,
    train_dataloader=train_dataloader,
    optimizer=optimizer,
    num_epochs=epochs,
    evaluator=evaluator,
    work_dir=work_dir
    )

## Start training

In [19]:
trainer.start(warmup_iters=100, checkpoints='best', select='max', validate_on='f1_score')

[TRAINING] - Epoch: [1] [  1/777] eta: 0:17:22 lr: 0.000002 loss: 12456.7910 (12456.7910) focal_loss: 12454.8086 (12454.8086) ce_loss: 1.9825 (1.9825) time: 1.3417 data: 0.8226 max mem: 7460
[TRAINING] - Epoch: [1] [ 51/777] eta: 0:09:31 lr: 0.000051 loss: 8600.5977 (10471.5576) focal_loss: 8598.9482 (10469.7762) ce_loss: 1.6718 (1.7814) time: 0.7700 data: 0.2795 max mem: 7668
[TRAINING] - Epoch: [1] [101/777] eta: 0:09:00 lr: 0.000100 loss: 1942.9307 (7134.0582) focal_loss: 1941.7195 (7132.4701) ce_loss: 1.2703 (1.5880) time: 0.7750 data: 0.2854 max mem: 7668
[TRAINING] - Epoch: [1] [151/777] eta: 0:08:14 lr: 0.000100 loss: 366.3689 (4983.3445) focal_loss: 365.3026 (4981.8661) ce_loss: 1.1080 (1.4785) time: 0.7497 data: 0.2599 max mem: 7668
[TRAINING] - Epoch: [1] [201/777] eta: 0:07:35 lr: 0.000100 loss: 143.8969 (3791.3712) focal_loss: 142.9811 (3789.9864) ce_loss: 0.9954 (1.3848) time: 0.8696 data: 0.3762 max mem: 7668
[TRAINING] - Epoch: [1] [251/777] eta: 0:06:53 lr: 0.000100 los

LossWrapper(
  (model): HerdNet(
    (base_0): DLA(
      (base_layer): Sequential(
        (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (level0): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (level1): Sequential(
        (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (level2): Tree(
        (tree1): BasicBlock(
          (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-0

## Test the model

In [20]:
# Path to your .pth file
import gdown

pth_path = '/home/christian/hnee/HerdNet/output/best_model.pth'

if not pth_path:
    gdown.download(
        'https://drive.google.com/uc?export=download&id=1-WUnBC4BJMVkNvRqalF_HzA1_pRkQTI_',
        '/content/20220413_herdnet_model.pth'
        )
    pth_path = '/content/20220413_herdnet_model.pth'

In [21]:
# Create output folder
test_dir = '../test_output'
mkdir(test_dir)

In [22]:
# Load trained parameters
from animaloc.models import load_model

herdnet = load_model(herdnet, pth_path=pth_path)

In [23]:
# Create an Evaluator
test_evaluator = HerdNetEvaluator(
    model=herdnet, 
    dataloader=test_dataloader, 
    metrics=metrics, 
    stitcher=stitcher, 
    work_dir=test_dir, 
    header='test'
    )

In [24]:
# Start testing
test_f1_score = test_evaluator.evaluate(returns='f1_score')

test [  1/258] eta: 0:21:35 n: 1 recall: 1.0 precision: 0.09 f1-score: 0.17 MAE: 10.0 MSE: 100.0 RMSE: 10.0 time: 5.0202 data: 0.6553 max mem: 7669
test [ 11/258] eta: 0:19:37 n: 1 recall: 1.0 precision: 0.1 f1-score: 0.18 MAE: 9.0 MSE: 81.0 RMSE: 9.0 time: 4.7471 data: 0.5361 max mem: 7669
test [ 21/258] eta: 0:18:51 n: 1 recall: 1.0 precision: 0.08 f1-score: 0.15 MAE: 11.0 MSE: 121.0 RMSE: 11.0 time: 4.7394 data: 0.5263 max mem: 7669
test [ 31/258] eta: 0:18:06 n: 36 recall: 0.94 precision: 0.77 f1-score: 0.85 MAE: 8.0 MSE: 64.0 RMSE: 8.0 time: 4.7786 data: 0.5283 max mem: 7669
test [ 41/258] eta: 0:17:27 n: 1 recall: 1.0 precision: 0.12 f1-score: 0.22 MAE: 7.0 MSE: 49.0 RMSE: 7.0 time: 4.8627 data: 0.5484 max mem: 7669
test [ 51/258] eta: 0:16:45 n: 6 recall: 1.0 precision: 0.46 f1-score: 0.63 MAE: 7.0 MSE: 49.0 RMSE: 7.0 time: 4.9326 data: 0.5689 max mem: 7669
test [ 61/258] eta: 0:16:01 n: 2 recall: 1.0 precision: 1.0 f1-score: 1.0 MAE: 0.0 MSE: 0.0 RMSE: 0.0 time: 4.9538 data: 0.

In [25]:
# Print global F1 score (%)
print(f"F1 score = {test_f1_score * 100:0.0f}%")

F1 score = 49%


In [26]:
# Get the detections
detections = test_evaluator.results
detections

,class,n,recall,precision,f1_score,confusion,mae,mse,rmse,ap
0,1,675,0.884444,0.714115,0.790205,0.067187,2.137615,11.293578,3.360592,0.809474
1,2,349,0.808023,0.546512,0.652023,0.140244,2.948052,19.779221,4.447384,0.666486
2,3,477,0.742138,0.692759,0.716599,0.257862,1.504132,7.090909,2.662876,0.681852
3,4,74,0.783784,0.107209,0.188618,0.000000,3.666667,23.821705,4.880748,0.373530
4,5,36,0.722222,0.412698,0.525253,0.235294,1.444444,2.851852,1.688743,0.604879
5,6,688,0.869186,0.152240,0.259099,0.000000,28.672566,3544.513274,59.535815,0.569560
6,binary,2299,0.928665,0.333855,0.491143,0.000000,15.891473,1604.705426,40.058775,0.587721
